In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle
import glob
import os

import sys
sys.path.append("../src")
import utils
from utils import text_processing
from text_processing import categorize_news

from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
import nltk
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier

pd.set_option("display.max_colwidth", None)


ModuleNotFoundError: No module named 'text_processing'

In [ ]:
categorize_news(df)